In [30]:
# Add one to the dates
# select first x characters then select the last one then add one to the day

In [1]:
# We use last 3 games so we can TRY to quantify the qualitative
# If we were to use last 10 game record:
#   team played well but lost
#   team played against hard team and lost
#   team played against bad team and won
# Using numbers allows us to get a better picture

In [2]:
# Pull team home game dates from 2005 to 2017
# Don't include 2017-2018 b/c of Vegas joining the league (or do not sure yet)
# Exclude pre-season
# Exclude playoffs

In [3]:
# To get last three games, first pull the scehdule of the team and then the corresponding stats
# Create a dictionary of dates and stats. then write a function to pull the last X days game stats

In [4]:
import pandas as pd
pd.set_option('display.max_colwidth', -1)
pd.set_option('display.max_rows', 500)
pd.set_option('display.max_columns', 500)
pd.set_option('display.width', 1000)

from urllib2 import Request, urlopen
import json

from pprint import *
pp = PrettyPrinter(indent=1)

In [5]:
def get_parsed_json(path):
    request = Request(path)
    response = urlopen(request)
    json_rsp = response.read()
    parsed_json = json.loads(json_rsp)
    return parsed_json

In [6]:
def is_reg_season_game(game_data):
    return True if str(game_data['games'][0]['gamePk'])[5] == '2' else False

In [7]:
def is_home_game(game, team_id):
    return True if game['games'][0]['teams']['home']['team']['id'] == team_id else False

In [8]:
def won_game(game):
    return 1 if game['games'][0]['teams']['home']['score'] > game['games'][0]['teams']['away']['score'] else 0

In [9]:
def get_team_api_stats(team_standings, team_id):
    
    team_stats = []
    
    # loop through conferences
    for i in range(6):
        # loop through games played on night
        for j in range(len(team_standings[i]['teamRecords'])):
            team_records = team_standings[i]['teamRecords'][j]
            if team_records['team']['id'] == team_id:
                team_stats.append(team_records['goalsScored'])
                team_stats.append(team_records['goalsAgainst'])
                team_stats.append(team_records['streak']['streakNumber'])
                team_stats.append(int(team_records['divisionRank']))
                team_stats.append(int(team_records['conferenceRank']))
                team_stats.append(int(team_records['leagueRank']))
                return team_stats

In [10]:
def get_team_nhl_stats(team_standings):

    team_stats = []
    
    team_stats.append(team_standings['ppPctg'] * 100)
    team_stats.append(team_standings['penaltyKillPctg'] * 100)
    team_stats.append(round(team_standings['goalsFor'] / float(team_standings['shotsFor']), 2) * 100)
    team_stats.append(round(team_standings['goalsAgainst'] / float(team_standings['shotsAgainst']), 2) * 100)
    team_stats.append(team_standings['faceoffWinPctg'] * 100)
    return team_stats

In [11]:
# Initialize dataframe

features = ['game_date', 'game_result', 'home_team_id','home_goals_for', 'home_goals_against', \
            'home_goal_differential', 'home_winning_streak', 'home_division_standing', \
            'home_conference_standing', 'home_league_standing', 'home_power_play_success_rate', \
            'home_power_kill_success_rate', 'home_shot_percentage', 'home_save_percentage', \
            'home_faceoff_win_percentage', 'home_fenwick_close_percentage', 'home_pdo', \
            'away_team_id','away_goals_for', 'away_goals_against', 'away_goal_differential', \
            'away_winning_streak', 'away_division_standing', 'away_conference_standing', 'away_league_standing', \
            'away_power_play_success_rate', 'away_power_kill_success_rate', 'away_shot_percentage', \
            'away_save_percentage', 'away_faceoff_win_percentage', 'away_fenwick_close_percentage', 'away_pdo']
df = pd.DataFrame(columns=features)

In [12]:
game_record = ["'{}':game_stat_lst[{}]".format(features[i], i) for i in range(len(features))]
game_record[0]

dicti = {i[1:] for i in game_record}
#dicti

In [13]:
def build_game_df(game_stat_lst):
    print(game_stat_lst)
    print(len(game_stat_lst))
    
    game_record = ["'{}':game_stat_lst[{}]".format(features[i], i) for i in range(len(features))]
    game_record

    dicti = {i for i in game_record}
    dicti
    
    game_df = pd.DataFrame(dicti, index=[0])
    return game_df

In [14]:
def build_game_df(game_stat_lst):
    print(game_stat_lst)
    print(len(game_stat_lst))
    game_df = pd.DataFrame({'game_date': game_stat_lst[0],
                            'game_result': game_stat_lst[1],
                            'home_team_id': game_stat_lst[2],
                            'home_goals_for': game_stat_lst[3],
                            'home_goals_against': game_stat_lst[4],
                            'home_goal_differential': game_stat_lst[5],
                            'home_winning_streak': game_stat_lst[6],
                            'home_division_standing': game_stat_lst[7],
                            'home_conference_standing': game_stat_lst[8],
                            'home_league_standing': game_stat_lst[9],
                            'home_power_play_success_rate': game_stat_lst[10],
                            'home_power_kill_success_rate': game_stat_lst[11],
                            'home_shot_percentage': game_stat_lst[12],
                            'home_save_percentage': game_stat_lst[13],
                            'home_faceoff_win_percentage': game_stat_lst[14],
                            'home_fenwick_close_percentage': game_stat_lst[15],
                            'home_pdo': game_stat_lst[16],
                            'away_team_id': game_stat_lst[17],
                            'away_goals_for': game_stat_lst[18],
                            'away_goals_against': game_stat_lst[19],
                            'away_goal_differential': game_stat_lst[20],
                            'away_winning_streak': game_stat_lst[21],
                            'away_division_standing': game_stat_lst[22],
                            'away_conference_standing': game_stat_lst[23],
                            'away_league_standing': game_stat_lst[24],
                            'away_power_play_success_rate': game_stat_lst[25],
                            'away_power_kill_success_rate': game_stat_lst[26],
                            'away_shot_percentage': game_stat_lst[27],
                            'away_save_percentage': game_stat_lst[28],
                            'away_faceoff_win_percentage': game_stat_lst[29],
                            'away_fenwick_close_percentage': game_stat_lst[30],
                            'away_pdo': game_stat_lst[31]},
                            index=[0])
    return game_df

In [43]:
for i in range(0,31):
    team_path = 'https://statsapi.web.nhl.com/api/v1/schedule?teamId=' + str(i) + '&startDate=2013-09-28&endDate=2018-04-25'
    team_data = get_parsed_json(team_path)

    num_of_games = team_data['totalGames']

    for j in range(0, num_of_games):

        game_data = team_data['dates'][j]
        
        if is_reg_season_game(game_data) and is_home_game(game_data, i) :
            game_id = game_data['games'][0]['gamePk']
            game_date = game_data['date']
            game_year = game_date[:4]
            game_result = won_game(game_data)
            home_team_id = i
            away_team_id = game_data['games'][0]['teams']['away']['team']['id']
            print "game_id:", game_id
            print "game_date:", game_date
            print "game_result:", game_result
            print "home_team_id:", i
            print "away_team_id:", away_team_id
            print('\n')

            # for when making API calls since game data isn't available until day after
            day_after_game = str(int(game_date[-1:]) + 1)
            date_after_game = game_date[:9] + day_after_game
            
            print "game_date:", game_date[:9]
            print "day_after_game", day_after_game
            print "date_after_game", date_after_game
            
            team_standings_path = 'https://statsapi.web.nhl.com/api/v1/standings?date=' + str(game_date)
            team_standings = get_parsed_json(team_standings_path)['records']
            home_api_stats = get_team_api_stats(team_standings, i)
            home_goals_for = home_api_stats[0]
            home_goals_against = home_api_stats[1]
            home_goal_differential = home_goals_for - home_goals_against
            home_winning_streak = home_api_stats[2]
            home_division_standing = home_api_stats[3]
            home_conference_standing = home_api_stats[4]
            home_league_standing = home_api_stats[5]
            
            home_nhl_stats_path = 'http://www.nhl.com/stats/rest/team?isAggregate=true&reportType=basic&isGame=true&reportName=teamsummary&sort=[{%22property%22:%22points%22,%22direction%22:%22DESC%22},{%22property%22:%22wins%22,%22direction%22:%22DESC%22}]&factCayenneExp=gamesPlayed%3E=1&cayenneExp=gameDate%3E=%22' + game_year + '-09-01%22%20and%20gameDate%3C=%22' + date_after_game + '%22%20and%20gameTypeId=2%20and%20teamId=' + str(i) + ''
            print(home_nhl_stats_path)
            home_nhl_stats_json = get_parsed_json(home_nhl_stats_path)
            home_nhl_stats = get_team_nhl_stats(home_nhl_stats_json['data'][0])
            home_power_play_success_rate = home_nhl_stats[0]
            home_power_kill_success_rate = home_nhl_stats[1]
            home_shot_percentage = home_nhl_stats[2]
            home_save_percentage = home_nhl_stats[3]
            home_faceoff_win_percentage = home_nhl_stats[4]

            home_sat_percentages_path = 'http://www.nhl.com/stats/rest/team?isAggregate=true&reportType=shooting&isGame=true&reportName=teampercentages&sort=[{%22property%22:%22shotAttemptsPctg%22,%22direction%22:%22DESC%22}]&factCayenneExp=gamesPlayed%3E=1&cayenneExp=gameDate%3E=%22' + game_year + '-09-01%22%20and%20gameDate%3C=%22' + date_after_game + '%22%20and%20gameTypeId=2%20and%20teamId=' + str(i) + ''
            home_sat_percentages = get_parsed_json(home_sat_percentages_path)['data'][0]
            home_fenwick_close_percentage = round(home_sat_percentages['unblockedShotAttemptsPctgClose'] * 100, 2)
            home_pdo = round((home_sat_percentages['shootingPlusSavePctg']) * 1000)

            print "home_goals_for:", home_goals_for
            print "home_goals_against:", home_goals_against
            print "home_goal_differential:", home_goal_differential
            print "home_winning_streak:", home_winning_streak
            print "home_division_standing:", home_division_standing
            print "home_conference_standing:", home_conference_standing
            print "home_league_standing:", home_league_standing
            print "home_power_play_success_rate:", home_power_play_success_rate
            print "home_power_kill_success_rate", home_power_kill_success_rate 
            print "home_shot_percentage", home_shot_percentage
            print "home_save_percentage", home_save_percentage
            print "home_faceoff_win_percentage", home_faceoff_win_percentage
            print "home_fenwick_close_percentage", home_fenwick_close_percentage
            print "home_pdo", home_pdo
            print('\n')

            away_api_stats = get_team_api_stats(team_standings, away_team_id)
            away_goals_for = away_api_stats[0]
            away_goals_against = away_api_stats[1]
            away_goal_differential = away_goals_for - away_goals_against
            away_winning_streak = away_api_stats[2]
            away_division_standing = away_api_stats[3]
            away_conference_standing = away_api_stats[4]
            away_league_standing = away_api_stats[5]
                     
            away_nhl_stats_path = 'http://www.nhl.com/stats/rest/team?isAggregate=true&reportType=basic&isGame=true&reportName=teamsummary&sort=[{%22property%22:%22points%22,%22direction%22:%22DESC%22},{%22property%22:%22wins%22,%22direction%22:%22DESC%22}]&factCayenneExp=gamesPlayed%3E=1&cayenneExp=gameDate%3E=%22' + game_year + '-09-01%22%20and%20gameDate%3C=%22' + date_after_game + '%22%20and%20gameTypeId=2%20and%20teamId=' + str(away_team_id) + ''
            away_nhl_stats_json = get_parsed_json(away_nhl_stats_path)
            away_nhl_stats = get_team_nhl_stats(away_nhl_stats_json['data'][0])
            pprint(away_nhl_stats)
            away_power_play_success_rate = away_nhl_stats[0]
            away_power_kill_success_rate = away_nhl_stats[1]
            away_shot_percentage = away_nhl_stats[2]
            away_save_percentage = away_nhl_stats[3]
            away_faceoff_win_percentage = away_nhl_stats[4]
            
            away_sat_percentages_path = 'http://www.nhl.com/stats/rest/team?isAggregate=true&reportType=shooting&isGame=true&reportName=teampercentages&sort=[{%22property%22:%22shotAttemptsPctg%22,%22direction%22:%22DESC%22}]&factCayenneExp=gamesPlayed%3E=1&cayenneExp=gameDate%3E=%22' + game_year + '-09-01%22%20and%20gameDate%3C=%22' + date_after_game + '%22%20and%20gameTypeId=2%20and%20teamId=' + str(away_team_id) + ''
            away_sat_percentages = get_parsed_json(away_sat_percentages_path)['data'][0]
            away_fenwick_close_percentage = round(away_sat_percentages['unblockedShotAttemptsPctgClose'] * 100, 2)
            away_pdo = round((away_sat_percentages['shootingPlusSavePctg']) * 1000)

            print "away_goals_for:", away_goals_for
            print "away_goals_against:", away_goals_against
            print "away_goal_differential:", away_goal_differential
            print "away_winning_streak:", away_winning_streak
            print "away_division_standing:", away_division_standing
            print "away_conference_standing:", away_conference_standing
            print "away_league_standing:", away_league_standing
            print "away_power_play_success_rate:", away_power_play_success_rate
            print "away_power_kill_success_rate", away_power_kill_success_rate 
            print "away_shot_percentage", away_shot_percentage
            print "away_save_percentage", away_save_percentage
            print "away_faceoff_win_percentage", away_faceoff_win_percentage
            print "away_fenwick_close_percentage", away_fenwick_close_percentage
            print "away_pdo", away_pdo
            print('\n')
            print('------------------------------------------\n')

            game_stat_lst = [game_date, game_result, i, home_goals_for, home_goals_against, home_goal_differential, \
                             home_winning_streak, home_division_standing, home_conference_standing, home_league_standing, \
                             home_power_play_success_rate, home_power_kill_success_rate, home_shot_percentage, \
                             home_save_percentage, home_faceoff_win_percentage, home_fenwick_close_percentage, home_pdo, \
                             away_team_id, away_goals_for, away_goals_against, away_goal_differential, \
                             away_winning_streak, away_division_standing, away_conference_standing, away_league_standing, \
                             away_power_play_success_rate, away_power_kill_success_rate, away_shot_percentage, \
                             away_save_percentage, away_faceoff_win_percentage, away_fenwick_close_percentage, away_pdo]

            game_df = build_game_df(game_stat_lst)

            df = df.append(game_df, ignore_index=True)

game_id: 2013020016
game_date: 2013-10-04
game_result: 0
home_team_id: 1
away_team_id: 2


game_date: 2013-10-0
day_after_game 5
date_after_game 2013-10-05
http://www.nhl.com/stats/rest/team?isAggregate=true&reportType=basic&isGame=true&reportName=teamsummary&sort=[{%22property%22:%22points%22,%22direction%22:%22DESC%22},{%22property%22:%22wins%22,%22direction%22:%22DESC%22}]&factCayenneExp=gamesPlayed%3E=1&cayenneExp=gameDate%3E=%222013-09-01%22%20and%20gameDate%3C=%222013-10-05%22%20and%20gameTypeId=2%20and%20teamId=1
home_goals_for: 3
home_goals_against: 7
home_goal_differential: -4
home_winning_streak: 1
home_division_standing: 5
home_conference_standing: 10
home_league_standing: 19
home_power_play_success_rate: 0.0
home_power_kill_success_rate 100.0
home_shot_percentage 5.0
home_save_percentage 13.0
home_faceoff_win_percentage 43.39
home_fenwick_close_percentage 54.29
home_pdo 917.0


[0.0, 100.0, 12.0, 10.0, 53.839999999999996]
away_goals_for: 4
away_goals_against: 3
away_goal_di

HTTPError: HTTP Error 500: Internal Server Error

In [ ]:
df